In [ ]:
from datetime import datetime
from time import time
import json
import logging

import tensorflow as tf
import keras
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras.models import Sequential
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.callbacks import Callback

from sklearn.metrics import r2_score

In [1]:
import tensorflow as tf

import pandas as pd
import codecs
import numpy as np
import math
import csv
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import recall_score, precision_score # New
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt
import tensorflow.keras.preprocessing
#from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten, Add, Multiply, Subtract
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, BatchNormalization
# regularizers
from tensorflow.keras.layers import Conv1D, Conv2D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dropout, SpatialDropout1D
from scipy import signal
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pylab as plt
import matplotlib.dates as mdates
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from numpy.linalg import svd
import gc
from keijzer import *
from tensorflow import keras
from keras.models import Model
from keras.layers import *
from tensorflow.keras.optimizers import Adam,SGD
from keras.layers import Dense, Input, Flatten, Add, concatenate, Dropout, Activation, Multiply, Embedding, Layer, Reshape
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D  
from keras.layers import Conv1D, AveragePooling1D, MaxPooling1D
#from keras.ops import convert_to_tensor, convert_to_numpy
#from keras.utils import plot_model
from keras import activations
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
import math

class LossHistory(tensorflow.keras.callbacks.Callback):  # history = LossHistory()
    def init(self):  # history.init()
        self.losses = []
        # self.accs = []
        self.val_losses = []
        # self.val_accs = []
        self.rmses = []
        self.mses = []
        self.maes = []
        self.mapes = []
        self.val_rmses = []
        self.val_mses = []
        self.val_maes = []
        self.val_mapes = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        # self.accs.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        # self.val_accs.append(logs.get('val_accuracy
        self.rmses.append(logs.get('root_mean_squared_error'))
        self.mses.append(logs.get('mean_squared_error'))
        self.maes.append(logs.get('mean_absolute_error'))
        self.mapes.append(logs.get('mean_absolute_percentage_error'))
        self.val_rmses.append(logs.get('val_root_mean_squared_error'))
        self.val_mses.append(logs.get('val_mean_squared_error'))
        self.val_maes.append(logs.get('val_mean_absolute_error'))
        self.val_mapes.append(logs.get('val_mean_absolute_percentage_error'))



def root_squared_mean_error(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true-K.mean(y_true)))*100

def mse_mae(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))+K.mean(K.abs(y_pred - y_true))

from keras.layers.advanced_activations import PReLU

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [2]:
import scipy as sc
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '0' # 여러개 사용시 '0,1,2' 식으로 하나의 문자열에 입력
gpus = tf.config.experimental.list_physical_devices('GPU') # 호스트 러나임에 표시되는 GPU 장치 목록 반환

if gpus: # 반환된 GPU 장치 목록이 있다면
    try: # 해당 장치에 대한 메모리 증가 활성화 여부 설정
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e: # try문 실패시에 에러문구 출력
        print(e)

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2020.csv'
data_source1 = pd.read_csv(data_path)
dat_source1 = pd.DataFrame(data_source1)
#dat_source1 = dat_source1.iloc[:-3,:]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2021.csv'
data_source2 = pd.read_csv(data_path)
dat_source2 = pd.DataFrame(data_source2)
dat_source2 = dat_source2.iloc[:8761,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2022.csv'
data_source3 = pd.read_csv(data_path)
dat_source3 = pd.DataFrame(data_source3)
dat_source3 = dat_source3.iloc[:8760,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2023.csv'
data_source4 = pd.read_csv(data_path)
dat_source4 = pd.DataFrame(data_source4)
dat_source4 = dat_source4.iloc[:8760,:9]

dat_arr1 = np.array(dat_source1)
dat_arr2 = np.array(dat_source2)
dat_arr3 = np.array(dat_source3)
dat_arr4 = np.array(dat_source4)

dat_arr = np.concatenate([dat_arr1, dat_arr2, dat_arr3, dat_arr4],axis=0)
dat_arr.shape # 8784, 8761, 8760, 8760

dat_source = pd.DataFrame(dat_arr, columns=['Time-Date stamp','Date','ERCOT Load','Total Wind Output','Total Wind Installed','Wind Output, % of Load','Wind Output, % of Installed','1-hr MW change','1-hr % change'])
dat = pd.DataFrame()
dat['MW % change'] = np.array(dat_source.iloc[1:,3], dtype=float)-np.array(dat_source.iloc[0:-1,3], dtype=float)
dat['MW % change'] = np.divide(np.array(dat['MW % change']),np.array(dat_source.iloc[1:,4]))*100
dat['Total Wind Output'] = np.divide(np.array(dat_source.iloc[1:,3]), np.array(dat_source.iloc[1:,4]))

ramp_ratio = 20
load_ratio = 10
year_arr = np.concatenate([np.ones(8784)*2020, np.ones(8760)*2021, np.ones(8760)*2022, np.ones(8760)*2023])
year_df = pd.DataFrame(year_arr, columns=['Year'], dtype=int)
wind_df = pd.concat([year_df, dat], axis=1)
wind_df = wind_df.reset_index(drop=True)

std_scaler = sklearn.preprocessing.MinMaxScaler()
std_scaler_load = sklearn.preprocessing.MinMaxScaler()
std_scaler_ramp = sklearn.preprocessing.MinMaxScaler()

wind_df[['Total Wind Output']] = std_scaler.fit_transform(wind_df[['Total Wind Output']])
wind_df[['MW % change']] = std_scaler_ramp.fit_transform(wind_df[['MW % change']])
wind_df = wind_df.iloc[:-1,:]
ramp_df = pd.DataFrame(wind_df)
ramp_df.columns=['Year', 'Wind Change', 'Wind Output']
ramp_df

,Year,Wind Change,Wind Output
0,2020,0.620197,0.644724
1,2020,0.544580,0.658617
2,2020,0.570751,0.683924
3,2020,0.599594,0.721813
4,2020,0.495246,0.714187
...,...,...,...
35058,2023,0.774643,0.484984
35059,2023,0.735512,0.582157
35060,2023,0.601593,0.620917
35061,2023,0.536583,0.631322


In [4]:
def create_dataset(dataset, timesteps, output_timesteps, leadtime):
    dataX, dataY = [], []
    for i in range(len(dataset) - timesteps -output_timesteps - leadtime - 1):
        a = dataset[i:(i + timesteps), :]
        dataX.append(a)
        dataY.append(dataset[(i + timesteps + leadtime):(i+timesteps+output_timesteps+leadtime), :])
    return np.array(dataX), np.array(dataY)

def svdadd(X, start, end, b_size):
    dec_x = X[start:end, :, :]
    U, S, Vh = svd(dec_x.reshape(b_size, -1), full_matrices=True)

    high_sig = np.matmul(np.matmul(U[:,:high_ind], np.diag(S[0:high_ind])), Vh[:high_ind,:])
    low_sig = X[:dec_num, :, :].reshape(b_size,-1)-high_sig

    rec_x = np.zeros((b_size, b_size, high_ind))
    
    for i in range(high_ind):
        rec_x[:,:,i] = np.matmul((U[:,i]*S[i]).reshape(-1,1), Vh[i,:].reshape(1,-1))    
    return rec_x, low_sig

In [5]:
norm_df = np.array(wind_df.iloc[:,1]).reshape(-1,1)
timesteps = 24*7
output_timesteps = 24
leadtime = 6
num_features = 1
X, Y = create_dataset(norm_df, timesteps, output_timesteps, leadtime)
b_size = timesteps
#X

In [6]:
dec_num = timesteps
high_ind = 35

addX = np.zeros((X.shape[0], dec_num, high_ind+1))

for k in range(X.shape[0]//dec_num):
    rec_x, low_sig = svdadd(X, k*dec_num, (k+1)*dec_num, b_size)
    addX[k*dec_num:k*dec_num+dec_num,:, :-1] = rec_x
    addX[k*dec_num:k*dec_num+dec_num,:, -1] = low_sig
    
rec_x, low_sig = svdadd(X, X.shape[0]-dec_num, X.shape[0], b_size)
addX[X.shape[0]-dec_num:X.shape[0], :, :-1] = rec_x
addX[X.shape[0]-dec_num:X.shape[0],:, -1] = low_sig

addX_det = addX[:, :, :-1]
addX_det = np.sum(addX_det, axis=2)
X = np.concatenate([X, addX], axis=2)
num_features = X.shape[2]

In [7]:
trvaX, teX, trvaY, teY = train_test_split(X, Y, test_size=0.1, shuffle=False)

idx = np.arange(trvaX.shape[0])
np.random.shuffle(idx)
strvaX = trvaX[idx]
strvaY = trvaY[idx]
strvaY = strvaY.reshape(-1, output_timesteps)

trvaY = trvaY.reshape(-1, output_timesteps)
teY = teY.reshape(-1, output_timesteps)

strX, svaX, strY, svaY = train_test_split(strvaX, strvaY, test_size=0.3, shuffle=False)
trX, vaX, trY, vaY = train_test_split(trvaX, trvaY, test_size=0.3, shuffle=False)

trvaX.shape, trvaY.shape, teX.shape, teY.shape

((31377, 168, 37), (31377, 24), (3487, 168, 37), (3487, 24))

In [8]:
def MAEMS(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true))*100

def MAEMD(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true-K.mean(y_true)))*100

def MAE(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true)))

def MSE(y_true, y_pred):
    return K.mean((K.square(y_pred - y_true)))

## Informer Model - With SVD With Shuffling

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, Model
class SelfAttention(layers.Layer):
    def __init__(self, d_model, n_heads):
        super(SelfAttention, self).__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.depth = d_model // n_heads

        self.wq = layers.Dense(d_model)
        self.wk = layers.Dense(d_model)
        self.wv = layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.n_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]

        q = self.wq(inputs)
        k = self.wk(inputs)
        v = self.wv(inputs)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention_logits = tf.matmul(q, k, transpose_b=True)
        scaled_attention_logits /= tf.math.sqrt(tf.cast(self.depth, tf.float32))

        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        
        output = tf.transpose(output, perm=[0, 2, 1, 3])
        output = tf.reduce_mean(output, axis=2)
        #print(output.shape)
        output = tf.reshape(output, (batch_size, -1, int(self.d_model / n_heads)))
        #print(output.shape)
        return output

class EncoderLayer(layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, rate=0.1):
        super(EncoderLayer, self).__init__()
        self.attention = SelfAttention(d_model, n_heads)
        self.ffn = tf.keras.Sequential([
            layers.Dense(d_ff, activation='relu'),
            layers.Dense(d_model // n_heads) 
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.attention(inputs)
        out1 = self.layernorm1(inputs + self.dropout1(attn_output, training=training))
        
        ffn_output = self.ffn(out1)
        #print(ffn_output.shape)
        return self.layernorm2(out1 + self.dropout2(ffn_output, training=training))

class Informer(Model):
    def __init__(self, input_shape, num_layers, d_model, n_heads, d_ff, output_length):
        super(Informer, self).__init__()
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff) for _ in range(num_layers)]
        self.flatten = layers.Flatten()  # Flatten before the final Dense layer
        self.final_layer = layers.Dense(output_length)  # Ensure output_length matches 24

    def call(self, inputs):
        x = inputs
        for layer in self.enc_layers:
            x = layer(x)
            
        x = self.flatten(x)  # Flatten the output
        return self.final_layer(x)

In [10]:
input_shape = (168, num_features)  # None for num_feature
num_layers = 20
d_model = 37*4
n_heads = 4
d_ff = n_heads*2
output_shape = 24

with tf.device('/gpu:0'):
    model = Informer(input_shape=input_shape, num_layers=num_layers, d_model=d_model, 
                      n_heads=n_heads, d_ff=d_ff, output_length=output_shape)

    batch_size = 168
    example_input = tf.random.normal((batch_size, *input_shape))

    output = model(example_input)
    model.summary()

Model: "informer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_layer (EncoderLayer) multiple                  17657     
_________________________________________________________________
encoder_layer_1 (EncoderLaye multiple                  17657     
_________________________________________________________________
encoder_layer_2 (EncoderLaye multiple                  17657     
_________________________________________________________________
encoder_layer_3 (EncoderLaye multiple                  17657     
_________________________________________________________________
encoder_layer_4 (EncoderLaye multiple                  17657     
_________________________________________________________________
encoder_layer_5 (EncoderLaye multiple                  17657     
_________________________________________________________________
encoder_layer_6 (EncoderLaye multiple                  176

In [11]:
gc.collect()

451

In [12]:
    model.compile(loss=MAEMD, optimizer='adam', metrics=['mse','mae', MAEMD])
    early_stopping =EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    batch_size = 168
    epochs = 1000
    history = LossHistory()
    history.init()

In [13]:
    %%time
    b_size = 168
    hist = model.fit(strX, strY, epochs=epochs, batch_size=b_size, shuffle=False, validation_data=(svaX, svaY), callbacks=[history, early_stopping])  # , checkpoint

Epoch 1/1000
131/131 [==============================] - 59s 321ms/step - loss: 0.4975 - mse: 0.4264 - mae: 0.4543 - MAEMD: 0.4972 - val_loss: 0.3286 - val_mse: 0.1246 - val_mae: 0.2900 - val_MAEMD: 0.3280
Epoch 2/1000
131/131 [==============================] - 39s 296ms/step - loss: 0.3109 - mse: 0.1071 - mae: 0.2595 - MAEMD: 0.3108 - val_loss: 0.2330 - val_mse: 0.0451 - val_mae: 0.1723 - val_MAEMD: 0.2316
Epoch 3/1000
131/131 [==============================] - 38s 293ms/step - loss: 0.2538 - mse: 0.0584 - mae: 0.1923 - MAEMD: 0.2537 - val_loss: 0.2242 - val_mse: 0.0381 - val_mae: 0.1551 - val_MAEMD: 0.2237
Epoch 4/1000
131/131 [==============================] - 39s 294ms/step - loss: 0.2233 - mse: 0.0352 - mae: 0.1488 - MAEMD: 0.2233 - val_loss: 0.2091 - val_mse: 0.0256 - val_mae: 0.1289 - val_MAEMD: 0.2086
Epoch 5/1000
131/131 [==============================] - 39s 297ms/step - loss: 0.2120 - mse: 0.0270 - mae: 0.1298 - MAEMD: 0.2120 - val_loss: 0.2067 - val_mse: 0.0249 - val_mae: 0.

In [14]:
gc.collect()

4594

In [15]:
testPredict = model.predict(teX, batch_size=b_size)

In [16]:
def npMAEMS(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true))*100
def npMAEMD(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true-np.mean(y_true)))*100
def npMSE(y_true, y_pred):
    return np.mean(np.square(-y_true+y_pred))
def npMAE(y_true, y_pred):
    return np.mean(abs(-y_true+y_pred))

In [17]:
tePredict = testPredict.reshape(-1)
testY = teY.reshape(-1)
print('Error Test Score > MSE == ', npMSE(testY, tePredict), ' MAE == ', npMAE(testY, tePredict), ' MAEMD == ', npMAEMD(testY, tePredict))

Error Test Score > MSE ==  0.010810348994591984  MAE ==  0.08211944045758982  MAEMD ==  0.10642939353682926


## Without Shuffling

In [103]:
input_shape = (168, num_features)  # None for num_feature
num_layers = 10
d_model = 37*4
n_heads = 4
d_ff = n_heads*4
output_shape = 24

with tf.device('/gpu:0'):
    model2 = Informer(input_shape=input_shape, num_layers=num_layers, d_model=d_model, 
                      n_heads=n_heads, d_ff=d_ff, output_length=output_shape)

    batch_size = 168
    example_input = tf.random.normal((batch_size, *input_shape))

    output = model2(example_input)
    model2.summary()

Model: "informer_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_layer_126 (EncoderLa multiple                  18257     
_________________________________________________________________
encoder_layer_127 (EncoderLa multiple                  18257     
_________________________________________________________________
encoder_layer_128 (EncoderLa multiple                  18257     
_________________________________________________________________
encoder_layer_129 (EncoderLa multiple                  18257     
_________________________________________________________________
encoder_layer_130 (EncoderLa multiple                  18257     
_________________________________________________________________
encoder_layer_131 (EncoderLa multiple                  18257     
_________________________________________________________________
encoder_layer_132 (EncoderLa multiple                  

In [104]:
gc.collect()

3445

In [105]:
    model2.compile(loss=MAEMD, optimizer='adam', metrics=['mse','mae', MAEMD])
    early_stopping =EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    batch_size = 168
    epochs = 1000
    history2 = LossHistory()
    history2.init()

In [106]:
    %%time
    hist2 = model2.fit(trX, trY, epochs=epochs, batch_size=b_size, shuffle=False, validation_data=(vaX, vaY), callbacks=[history2, early_stopping])  # , checkpoint

Epoch 1/1000
131/131 [==============================] - 30s 163ms/step - loss: 0.5699 - mse: 0.5432 - mae: 0.4812 - MAEMD: 0.5693 - val_loss: 0.2248 - val_mse: 0.0604 - val_mae: 0.2059 - val_MAEMD: 0.2264
Epoch 2/1000
131/131 [==============================] - 20s 151ms/step - loss: 0.3076 - mse: 0.0931 - mae: 0.2399 - MAEMD: 0.3074 - val_loss: 0.1941 - val_mse: 0.0527 - val_mae: 0.1922 - val_MAEMD: 0.1962
Epoch 3/1000
131/131 [==============================] - 20s 150ms/step - loss: 0.2447 - mse: 0.0619 - mae: 0.1969 - MAEMD: 0.2445 - val_loss: 0.1560 - val_mse: 0.0345 - val_mae: 0.1528 - val_MAEMD: 0.1586
Epoch 4/1000
131/131 [==============================] - 20s 150ms/step - loss: 0.2111 - mse: 0.0447 - mae: 0.1668 - MAEMD: 0.2109 - val_loss: 0.1351 - val_mse: 0.0235 - val_mae: 0.1224 - val_MAEMD: 0.1371
Epoch 5/1000
131/131 [==============================] - 20s 150ms/step - loss: 0.1900 - mse: 0.0343 - mae: 0.1467 - MAEMD: 0.1898 - val_loss: 0.1173 - val_mse: 0.0192 - val_mae: 0.

In [107]:
gc.collect()

704538

In [108]:
testPredict2 = model2.predict(teX, batch_size=b_size)

In [109]:
tePredict2 = testPredict2.reshape(-1)
testY = teY.reshape(-1)
print('Error Test Score > MSE == ', npMSE(testY, tePredict), ' MAE == ', npMAE(testY, tePredict), ' MAEMD == ', npMAEMD(testY, tePredict))
print('Error Test Score > MSE == ', npMSE(testY, tePredict2), ' MAE == ', npMAE(testY, tePredict2), ' MAEMD == ', npMAEMD(testY, tePredict2))

Error Test Score > MSE ==  0.010810348994591984  MAE ==  0.08211944045758982  MAEMD ==  0.10642939353682926
Error Test Score > MSE ==  0.011241171126035657  MAE ==  0.08322160423327984  MAEMD ==  0.0949018921183549


In [110]:
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import norm

def diebold_mariano_test(forecast1, forecast2, actual, opt): 
    
    if opt==0: # MSE
        e1 = actual-forecast1
        e2 = actual-forecast2
        d = e1**2 - e2**2
    elif opt==1: # MAE
        e1 = abs(actual-forecast1)
        e2 = abs(actual-forecast2)
        d = e1 - e2
    else:
        e1 = np.multiply(abs(actual - forecast1), (actual-np.mean(actual))**2)
        e2 = np.multiply(abs(actual - forecast2), (actual-np.mean(actual))**2)
        d = e1-e2
    
    # Mean of the loss differentials
    mean_d = np.mean(d)
    
    # Standard deviation of the loss differentials
    std_d = np.std(d, ddof=1)
    
    # Calculate the test statistic
    test_stat = (mean_d / std_d) * np.sqrt(len(d))
    
    # Calculate the p-value using a two-tailed test
    p_value = 2 * (1 - norm.cdf(abs(test_stat)))
    
    return test_stat, p_value

In [111]:
diebold_mariano_test(testPredict, testPredict2, teY, 0), diebold_mariano_test(testPredict, testPredict2, teY, 1), diebold_mariano_test(testPredict, testPredict2, teY, 2)

((-1.5409780152539156, 0.12332213672026038),
 (-0.964647417570333, 0.33472144064198583),
 (4.536865189273556, 5.709657849406469e-06))